In [2]:
!pip install --upgrade matplotlib > None 

In [3]:
# need refresh enviroment after installation

# Подключение библиотек

In [18]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor

# Модель. Подготовка.

In [5]:
# Dataset
df = pd.read_csv('winequality-white.csv', sep=';')
df.columns = [i.capitalize() for i in list(df.columns)]
df = df.rename({'Ph': 'pH'}, axis=1)

In [6]:
df.head()

,Fixed acidity,Volatile acidity,Citric acid,Residual sugar,Chlorides,Free sulfur dioxide,Total sulfur dioxide,Density,pH,Sulphates,Alcohol,Quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [7]:
X_array = df[df.columns[:-1]] 
Y_array = df[['Quality']]

Разделение на обучающую (75%) и тестовую (25%) выборки

In [8]:
X_array_train, X_array_test, Y_array_train, Y_array_test = train_test_split(X_array, Y_array, random_state=2, test_size=0.25) 
X_array_train.shape, X_array_test.shape, Y_array_train.shape, Y_array_test.shape

((3673, 11), (1225, 11), (3673, 1), (1225, 1))

# 1. Построить более сложную модель с подбором гиперпараметров

### Подбор и оптимизация гиперпараметров с использованием кросс-валидации. Grid Search

#### Создаем пайплайн

In [9]:
regr = Pipeline([("scaler", StandardScaler()), ("reg", RandomForestRegressor(random_state=0)),])

Тестируем по без настройки гиперпараметров

In [10]:
model = regr.fit(X_array_train, Y_array_train)
Y_test_prediction = model.predict(X_array_test)
print(f'MSE: {mean_squared_error(Y_array_test, Y_test_prediction)}')

MSE: 0.3882280816326531


Настройка гиперпараметров

In [35]:
parameters = {
    'reg__n_estimators': range(1, 102, 10), # Задали произвольно
    'reg__max_depth': range(1, 25, 3), # Задали произвольно
}

gridregr = GridSearchCV(regr, parameters, cv=5)
gridregr.fit(X_array_train, Y_array_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('reg',
                                        RandomForestRegressor(random_state=0))]),
             param_grid={'reg__max_depth': range(1, 25, 3),
                         'reg__n_estimators': range(1, 102, 10)})

In [34]:
best_parameters = gridregr.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(f"{param_name}: {best_parameters[param_name]}") #Наилучшие параметры

reg__max_depth: 21
reg__n_estimators: 99


In [13]:
y_pred_optimized = gridregr.best_estimator_.predict(X_array_test)
print(f'MSE: {mean_squared_error(Y_array_test, y_pred_optimized)}')

MSE: 0.3874397006953881


Ошибка немного уменьшилась.

### Выбор более сложной ML-модели

Выберем технику ансамблирования моделей с целью улучшения качества предсказаний оценки качества вина.

Ансамблирование

In [32]:
models = [LinearRegression(), Ridge(), KNeighborsRegressor()]
preds = []
for model in models:
  model.fit(X_array_train, Y_array_train)
  preds.append(model.predict(X_array_test))
y_preds = np.array(preds).mean(axis=0)
print(f'Hyper MSE: {mean_squared_error(Y_array_test, y_preds)}')

Hyper MSE: 0.5245111689220264


Результат лучше, чем у линейной регрессии из первого ДЗ, но не значительно.

## 2. Проинтепретировать результаты

В рамках первого ДЗ с помощью линейной регрессии была получена точность 0.5765523771614226.

Проведем обучение на линейной регрессии с подобранными гиперпараметрами.


In [33]:
linear_reg = LinearRegression()

linear_reg.fit(X_array_train, Y_array_train)

Y_test_prediction = linear_reg.predict(X_array_test)
print("Mean absolute error (LinearRegression-test): {:.7f}".format(mean_absolute_error(Y_array_test, Y_test_prediction)))

Mean absolute error (LinearRegression-test): 0.5858426


Ошибка осталась на прежнем уровне. 

Наилучши результат показал RandomForestRegressor с MSE: 0.3874397006953881 с оптимальной настройкой гиперпараметров.